### 记忆化递归 O(n)

In [8]:
cache = {}
def fib(n):
    if n in cache:
        return cache[n]
    
    if n <= 1:
        return n
    
    cache[n] = fib(n - 2) + fib(n - 1)
    return cache[n]

In [9]:
fib(100)

354224848179261915075

In [10]:
%timeit fib(100)

321 ns ± 8.74 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [35]:
%timeit fib(1000)

14 µs ± 886 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### 迭代 O(n)

In [6]:
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

In [7]:
fib(100)

354224848179261915075

In [11]:
%timeit fib(100)

338 ns ± 45.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### LRU cache

In [14]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n <= 1:
        return n
    return fib(n - 1) + fib(n - 2)

In [16]:
fib(100)

354224848179261915075

In [15]:
%timeit fib(100)

197 ns ± 40.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### 函数式 O(n)

In [22]:
from itertools import tee, islice
from operator import add

def fibs():
    yield 0 
    yield 1 
    fibs1, fibs2 = tee(fibs(), 2)
    yield from map(add, fibs1, islice(fibs2, 1, None))

In [25]:
%%timeit 

itr = fibs()
for i in range(101):
    num = next(itr)

1.78 ms ± 78 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
print(num)

354224848179261915075


In [27]:
from itertools import accumulate, repeat

def iterate(f, x):
    return accumulate(repeat(x), lambda fx, _: f(fx))

def next_fib(pair):
    x, y = pair
    return (y, x + y)

def fibs():
    return (y for x, y in iterate(next_fib, (0, 1)))

In [30]:
%%timeit 
itr = fibs()
for i in range(101):
    num = next(itr)

92 µs ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [31]:
print(num)

354224848179261915075


### 快速幂 O(log[n])

In [39]:
def fib(n):
    assert(n >= 0)
    return _fib(n)[0]


def _fib(n):
    if n == 0:
        return (0, 1)

    a, b = _fib(n // 2)
    c = a * (2 * b - a)
    d = a * a + b * b
    if n % 2 == 0:
        return (c, d)
    return (d, c + d)

In [40]:
%timeit fib(100)

8.54 µs ± 921 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
%timeit fib(1000)

14.7 µs ± 984 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Matrix exponentiation

$$
\left[
\begin{matrix}
F(n+1) & F(n) \\
F(n) & F(n-1) \\
\end{matrix}
\right]
=
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]^n
$$

数学归纳法证明：

1. $n = 1$, 有

$$ 
\left[
\begin{matrix}
F(2) & F(1) \\
F(1) & F(0) \\
\end{matrix}
\right]
=
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]^1
$$

2. 假设 $n = k$ 时，有
$$
\left[
\begin{matrix}
F(k+1) & F(k) \\
F(k) & F(k-1) \\
\end{matrix}
\right]
=
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]^k
$$
那么
$$
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]^{k+1}
= 
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]^{k}
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]
= 
\left[
\begin{matrix}
F(k+1) & F(k) \\
F(k) & F(k-1) \\
\end{matrix}
\right]
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]
= 
\left[
\begin{matrix}
F(k+1) + F(k) & F(k+1) \\
F(k) + F(k-1) & F(k) \\
\end{matrix}
\right]
= 
\left[
\begin{matrix}
F(k+2) & F(k+1) \\
F(k+1) & F(k) \\
\end{matrix}
\right]
$$
所以 $n = k+1$ 时依然成立.

$$
\left[
\begin{matrix}
F(2n+1) & F(n) \\
F(2n)  & F(2n-1) \\
\end{matrix}
\right]
=
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]^{2n}
=
(
\left[
\begin{matrix}
1 & 1 \\
1 & 0 \\
\end{matrix}
\right]^{n})^2
= 
\left[
\begin{matrix}
F(n+1) & F(2n) \\
F(n) & F(n-1) \\
\end{matrix}
\right]^{2}
= 
\left[
\begin{matrix}
F(n+1)^2+F(n)^2 & F(n+1)F(n)+F(n)F(n-1) \\
F(n+1)F(n)+F(n)F(n-1) & F(n)^2+F(n-1)^2 \\
\end{matrix}
\right]^{2n}
$$

所以

$$
F(2n + 1) =  F(n+1)^2 + F(n)^2;
$$

$$
F(2n) =  F(n+1)F(n) + F(n)F(n-1) = F(n+1)F(n) + F(n) (F(n+1) - F(n)) = F(n)(2F(n+1)-F(n))
$$



```python
def _fib(n):
    if n == 0:
        return (0, 1)

    a, b = _fib(n // 2)
    c = a * (2 * b - a)
    d = a ** 2 + b ** 2
    if n % 2 == 0:
        return (c, d)
    return (d, c + d)
```

$$
F(2n) = F(n) (2F(n+1) - F(n))
$$ 

$$
F(2n+1) = F(n+1)^2 + F(n)^2
$$